## Chat Models - Output Parsing


Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.

Output parsers are classes that help structure language model responses. There are two main methods an output parser must implement:

"Get format instructions": A method which returns a string containing instructions for how the output of a language model should be formatted.
"Parse": A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.
And then one optional one:

"Parse with prompt": A method which takes in a string (assumed to be the response from a language model) and a prompt (assumed to the prompt that generated such a response) and parses it into some structure. The prompt is largely provided in the event the OutputParser wants to retry or fix the output in some way, and needs information from the prompt to do so.


In [ ]:
%pip install langchain langchain_openai --upgrade

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'API_KEY_HERE'

In [1]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai.chat_models import ChatOpenAI

from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [2]:
# chat = ChatOpenAI(openai_api_key="...")

# If you have an envionrment variable set for OPENAI_API_KEY, you can just do:
chat = ChatOpenAI(temperature=0)

In [3]:
from typing import List


class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")


class Jokes(BaseModel):
    jokes: List[Joke] = Field(description="list of jokes")

In [4]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Jokes)

In [5]:
template = "Answer the user query.\n{format_instructions}\n{query}\n"
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt])

In [6]:
parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"$defs": {"Joke": {"properties": {"setup": {"description": "question to set up a joke", "title": "Setup", "type": "string"}, "punchline": {"description": "answer to resolve the joke", "title": "Punchline", "type": "string"}}, "required": ["setup", "punchline"], "title": "Joke", "type": "object"}}, "properties": {"jokes": {"description": "list of jokes", "items": {"$ref": "#/$defs/Joke"}, "title": "Jokes", "type": "array"}}, "required": ["jokes"]}\n```'

In [12]:
# Format the chat prompt:
messages = chat_prompt.format_prompt(
    format_instructions=parser.get_format_instructions(),
    query="What's really funny about Python programming?",
).to_messages()

In [13]:
result = chat.invoke(messages)

In [14]:
print(result.content)

{"jokes": [{"setup": "What's really funny about Python programming?", "punchline": "IndentationError: unexpected indent"}]}


In [15]:
joke_pydantic_object = parser.parse(result.content)

In [16]:
joke_pydantic_object.dict()

C:\Users\rgri\AppData\Local\Temp\ipykernel_12824\1662641841.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  joke_pydantic_object.dict()


{'jokes': [{'setup': "What's really funny about Python programming?",
   'punchline': 'IndentationError: unexpected indent'}]}

In [17]:
joke_pydantic_object.jokes

[Joke(setup="What's really funny about Python programming?", punchline='IndentationError: unexpected indent')]

In [18]:
joke_pydantic_object.jokes[0].punchline

'IndentationError: unexpected indent'